Purpose: Crawl Yelp's top 10 Gainesville restaurants and rate the sentiment of the comments

In [131]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import nltk
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import bigrams
from nltk.collocations import *
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [132]:
driver = webdriver.Chrome('chromedriver.exe')

C:\Users\cashe\AppData\Local\Temp/ipykernel_31244/530843676.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [110]:
driver.get("https://www.yelp.com/search?find_loc=Gainesville%2C+FL&start=0")

In [111]:
# get the first 10 popular restaurants' webpages
pages=[]
driver.get("https://www.yelp.com/search?find_loc=Gainesville%2C+FL&start=0")
box = driver.find_elements_by_class_name("businessName__09f24__EYSZE.display--inline-block__09f24__fEDiJ.border-color--default__09f24__NPAKY")
allLinks = []
for j in range(len(box)):
    allLinks.extend(box[j].find_elements_by_tag_name("a"))
Links = list(set([link.get_attribute("href") for link in allLinks]))
for link1 in Links:
    if link1.startswith('https://www.yelp.com/biz/'):
        pages.append(link1)
pages

C:\Users\cashe\AppData\Local\Temp/ipykernel_31244/2370235309.py:4: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  box = driver.find_elements_by_class_name("businessName__09f24__EYSZE.display--inline-block__09f24__fEDiJ.border-color--default__09f24__NPAKY")
C:\Users\cashe\AppData\Local\Temp/ipykernel_31244/2370235309.py:7: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  allLinks.extend(box[j].find_elements_by_tag_name("a"))


['https://www.yelp.com/biz/sweet-dreams-of-gainesville-gainesville',
 'https://www.yelp.com/biz/blaze-pizza-gainesville',
 'https://www.yelp.com/biz/the-top-gainesville',
 'https://www.yelp.com/biz/dragonfly-sushi-and-sake-gainesville-2',
 'https://www.yelp.com/biz/la-tienda-latina-gainesville',
 'https://www.yelp.com/biz/bolay-gainesville',
 'https://www.yelp.com/biz/maple-street-biscuit-gainesville-gainesville',
 'https://www.yelp.com/biz/metro-diner-gainesville',
 'https://www.yelp.com/biz/satchels-pizza-gainesville',
 'https://www.yelp.com/biz/beque-holic-gainesville']

In [112]:
info = []
reviews = []

# go to each page and get the Restaurant Name, Rating and Reviews
for link in range(len(pages)):
    driver.get(pages[link])
    time.sleep(1)
    
    title = driver.find_element_by_class_name("css-dyjx0f")
    title.text
    
    try:
        r = driver.find_element_by_class_name("i-stars__09f24__M1AR7.i-stars--large-4__09f24__F5sQy.border-color--default__09f24__NPAKY.overflow--hidden__09f24___ayzG")
        rate = r.get_attribute("aria-label")
    except:
        r = driver.find_element_by_class_name("i-stars__09f24__M1AR7.i-stars--large-4-half__09f24__pXLBr.border-color--default__09f24__NPAKY.overflow--hidden__09f24___ayzG")
        rate = r.get_attribute("aria-label")
        
    for i in range(1,11):
    
        begin = '//*[@id="main-content"]/div[2]/section[2]/div[2]/div/ul/li['
        end1 = "]/div/div[3]/p/span"
        end2 = "]/div/div[4]/p/span"
    
        try:
            path = begin + str(i) +end1
            review = driver.find_elements_by_xpath(path)
            review_text = review[0].text
            reviews.append(review_text)
        
        except:
            path = begin + str(i) +end2
            review = driver.find_elements_by_xpath(path)
            review_text = review[0].text
            reviews.append(review_text)
        
        info.append([title.text,rate,review_text])

C:\Users\cashe\AppData\Local\Temp/ipykernel_31244/3684527985.py:7: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  title = driver.find_element_by_class_name("css-dyjx0f")
C:\Users\cashe\AppData\Local\Temp/ipykernel_31244/3684527985.py:11: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  r = driver.find_element_by_class_name("i-stars__09f24__M1AR7.i-stars--large-4__09f24__F5sQy.border-color--default__09f24__NPAKY.overflow--hidden__09f24___ayzG")
C:\Users\cashe\AppData\Local\Temp/ipykernel_31244/3684527985.py:14: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  r = driver.find_element_by_class_name("i-stars__09f24__M1AR7.i-stars--large-4-half__09f24__pXLBr.border-color--default__09f24__NPAKY.overflow--hidden__09f24___ayzG")
C:\Users\cashe\AppData\Local\Temp/ip

In [148]:
#Convert to dataframe
import pandas as pd
df = pd.DataFrame(info,columns =["Restaurant Name","Rating","Reviews"])

In [149]:
# load stopwords and punctuation
stopWords = set(stopwords.words('english'))
string.punctuation

import re

#remove stop words, punctuation and remove words starting with a digit
allwords = []
for i in range(len(info)):
    wordsFiltered = []
    words = re.sub(r'^\d+','',(info[i][2]))
    words = word_tokenize(words)
    for r in words:
        if r.lower() not in stopWords and r not in string.punctuation:
            wordsFiltered.append(r)
    allwords.append(wordsFiltered)

# Group the words by reviews
allsentences = []
for i in range(len(allwords)):
    s = ''
    for j in range(len(allwords[i])):
        sentence = allwords[i][j]
        s = s + sentence + " "
    allsentences.append(s)
print(allsentences[0:10])
print(len(allsentences))

["Sweet cravings satisfied Date night Sweet Dreams homemade ice cream Gainesville 's locally owned traditional ice cream parlor serves homemade ice cream classics unique flavors Many housemade flavors sweet tooth went Peanut butter Chocolate Cookie Cream Add chocolate-dipped waffle cone brownie prices worthy scoop sizes big without skimping stuck flavor try favorite Honey-Cinnamon Employees typically friendly Sit indoors chairs outside take selfie ornate Ice Cream Sculpture locals ice cream go-to spot ice creams Gainesville Sweet Dreams Westgate Regency Shopping Center behind PDQ plenty parking Sweet ", "sweet find husband new Gainesville stumbled across Yelpers positive reviews House Chocolate delicious recommended Brown Sugar Toffee chef 's kiss definitely return try flavors ", "OK ice cream n't think 'd back Prices good lot flavors ice cream pretty standard ", 'Amazing ice cream made in-house really feels like home perfect pick-me-up afternoon treat evening snack wide variety flavor

In [150]:
# def the function to make Sentiment Analysis

def sentiment_scores(sentence):
    positive  = []
    negative =[]
    neutral =[]
    overall = []
    
    for i in sentence:
        sid_obj = SentimentIntensityAnalyzer()
        sentiment_dict = sid_obj.polarity_scores(i)
        positive.append(sentiment_dict['pos'])
        negative.append(sentiment_dict['neg'])
        neutral.append(sentiment_dict['neu'])
        #print(sentiment_dict['compound'])

 
    # decide sentiment as positive, negative and neutral
        if sentiment_dict['compound'] >= 0.05 :
            attitude = "Positive"
            overall.append(attitude)
 
        elif sentiment_dict['compound'] <= - 0.05 :
            attitude = "Negative"
            overall.append(attitude)
 
        else :
            attitude = "Neutral"
            overall.append(attitude)
            
    #print(positive)
    #print(negative)
    #print(neutral)
    #print(overall)
    return (positive,negative,neutral,overall)

In [151]:
#call the function
positive,negative,neutral,overall = sentiment_scores(allsentences)
#Convert to dataframe
df['positive'] = pd.DataFrame(positive)
df['negative'] = pd.DataFrame(negative)
df['neutral'] = pd.DataFrame(neutral)
df['overall'] = pd.DataFrame(overall)
df

,Restaurant Name,Rating,Reviews,positive,negative,neutral,overall
0,Sweet Dreams of Gainesville,4.5 star rating,Sweet cravings satisfied! Date night at Sweet ...,0.331,0.000,0.669,Positive
1,Sweet Dreams of Gainesville,4.5 star rating,Such a sweet find! My husband and I are new to...,0.494,0.000,0.506,Positive
2,Sweet Dreams of Gainesville,4.5 star rating,OK ice cream but I don't think I'd be back her...,0.429,0.000,0.571,Positive
3,Sweet Dreams of Gainesville,4.5 star rating,Amazing ice cream made in-house that really fe...,0.486,0.000,0.514,Positive
4,Sweet Dreams of Gainesville,4.5 star rating,Awesome little local ice cream place! My first...,0.396,0.000,0.604,Positive
...,...,...,...,...,...,...,...
95,Beque Holic,4 star rating,You can't go wrong here. Also Sam is the BEST ...,0.404,0.118,0.478,Positive
96,Beque Holic,4 star rating,This place used to be one of my favorite resta...,0.101,0.226,0.673,Negative
97,Beque Holic,4 star rating,I've been twice. Once a few years ago and once...,0.296,0.000,0.704,Positive
98,Beque Holic,4 star rating,"Great quality of meat, great soup, rice, and m...",0.310,0.030,0.660,Positive


In [152]:
#write to csv
df.to_csv('yelp.csv')